# Replace Links


- This notebook is designed to replace any web-links for images with links to new, locally downloaded versions. 
- Goal is to make images show up in Markdown files on GitHub and in Jupyter Notebook.

- 08/26
- This is second attempt, after ChatGPT3.5 starter code failed to work when applied. It only partially worked.

### Reference
- Final Chat Link for Successful Functions: https://chat.openai.com/share/362b6386-bb66-4956-bf78-d07c33c413cd

In [14]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote

# Sanitize file paths for Windows
def sanitize_filepath(filepath):
    return re.sub(r'[<>:"/\\|?*]', '_', filepath)

# Modified download_image to save in the folder of the original file and check for existing files
def download_image(url, folder_path):
    filename = os.path.basename(unquote(url))
    sanitized_filename = sanitize_filepath(filename)
    local_filepath = os.path.join(folder_path, 'images', sanitized_filename)
    
    # Check if the file already exists locally
    if os.path.exists(local_filepath):
        return local_filepath
    
    try:
        img_data = requests.get(url).content
    except requests.exceptions.MissingSchema:
        print(f"Skipping invalid URL: {url}")
        return None
    
    with open(local_filepath, 'wb') as img_file:
        img_file.write(img_data)
    
    return local_filepath


# # Modified update_markdown to accept folder_path and handle HTML img tags
# def update_markdown(file_content, folder_path):
#     # Handle Markdown image links
#     markdown_pattern = r'!\[.*?\]\((.*?)\)'
#     markdown_matches = re.findall(markdown_pattern, file_content)
#     for img_url in markdown_matches:
#         local_filepath = download_image(img_url, folder_path)  
#         # Replace image URLs with local file paths in HTML format for GitHub compatibility
#         file_content = file_content.replace(img_url, f'<img src="{local_filepath}" />')
    
#     # Handle HTML image links
#     html_pattern = r'<img.*?src="(.*?)".*?>'
#     html_matches = re.findall(html_pattern, file_content)
#     for img_url in html_matches:
#         local_filepath = download_image(img_url, folder_path)
#         file_content = file_content.replace(img_url, local_filepath)
    
#     return file_content


# Modified update_markdown to accept folder_path and handle HTML img tags without nesting
def update_markdown(file_content, folder_path):
    # Handle Markdown image links
    markdown_pattern = r'!\[.*?\]\((.*?)\)'
    markdown_matches = re.findall(markdown_pattern, file_content)
    for img_url in markdown_matches:
        # Skip if it's already an HTML img tag
        if "<img" in img_url:
            continue

        local_filepath = download_image(img_url, folder_path)  
        # Replace image URLs with local file paths in HTML format for GitHub compatibility
        replacement = f'<img src="{local_filepath}" />'
        file_content = file_content.replace(f'![img]({img_url})', replacement)
    
    # Handle HTML image links
    html_pattern = r'<img.*?src="(.*?)".*?>'
    html_matches = re.findall(html_pattern, file_content)
    for img_url in html_matches:
        local_filepath = download_image(img_url, folder_path)
        file_content = file_content.replace(f'src="{img_url}"', f'src="{local_filepath}"')
    
    return file_content

# Update HTML files
def update_html(file_content):
    soup = BeautifulSoup(file_content, 'html.parser')
    for img_tag in soup.find_all('img'):
        img_url = img_tag.get('src')
        if img_url:
            local_filepath = download_image(img_url)
            img_tag['src'] = local_filepath
    
    return str(soup)

# Modified update_file function to include a suffix for the new file
def update_file(filepath, suffix="_updated",verbose=True):
    folder_path = os.path.dirname(filepath)
    images_folder_path = os.path.join(folder_path, 'images')
    original_filename = os.path.basename(filepath)
    name, ext = os.path.splitext(original_filename)
    new_filename = f"{name}{suffix}{ext}"
    new_filepath = os.path.join(folder_path, new_filename)

    # Create the images folder if it doesn't exist
    if not os.path.exists(images_folder_path):
        os.mkdir(images_folder_path)
    
    with open(filepath, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # Identify file type and update accordingly
    if filepath.endswith('.html'):
        updated_content = update_html(file_content)
    elif filepath.endswith('.md'):
        updated_content = update_markdown(file_content, folder_path)  # Include folder_path
    else:
        print('Unsupported file type.')
        return

    # Save the updated content to a new file
    with open(new_filepath, 'w', encoding='utf-8') as file:
        file.write(updated_content)
        
    if verbose:
        print(f"- File updated successfully. New file saved at: {new_filepath}")



In [15]:
# # Example usage
# update_file('example.html')
# update_file('example.md')

In [16]:
import glob
files = sorted(glob.glob("*23*.md"))
files = list(filter(lambda x: '_repo' not in x.lower(),files))
# files = [f for f in files in "updated" not in x.lower()]]
files 

['Installation Overview-v23.md',
 'Instructions-updating-dojo-env-v23.md',
 'instructions-mac-intel-v23.md',
 'instructions-mac-mchip-v23.md',
 'instructions-windows-v23.md']

In [17]:
# filepath = files[0]
# filepath

In [18]:
# filepath = "Installation Overview-v23.md"
# update_file(filepath)

In [19]:
for filepath in files:
    update_file(filepath, suffix='_Repo')

- File updated successfully. New file saved at: Installation Overview-v23_Repo.md
- File updated successfully. New file saved at: Instructions-updating-dojo-env-v23_Repo.md
- File updated successfully. New file saved at: instructions-mac-intel-v23_Repo.md
- File updated successfully. New file saved at: instructions-mac-mchip-v23_Repo.md
- File updated successfully. New file saved at: instructions-windows-v23_Repo.md


In [20]:
# # Testing naming
# filepath_no_ext, ext = os.path.splitext(filepath)
# filepath_out_suffix='-Final'
# filepath_out = f"{filepath_no_ext}{filepath_out_suffix}{ext}"
# filepath_out

In [21]:
# update_file(file)